In [2]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from keras.datasets import mnist

In [1]:
class myDense():
  def __init__(self, input_size, output_size, activation):
      self.activation = activation

      w_shape = (input_size, output_size)

      #nie wpadamy w pulapke symetrii???

      w_init = tf.random.uniform(w_shape, maxval = 1e-1, minval = 0)
      self.W = tf.Variable(w_init)

      b_shape = (output_size)
      b_init = tf.zeros(b_shape)
      self.b= tf.Variable(b_init)

  def __call__(self, inputs):
    return self.activation(tf.matmul(inputs, self.W) + self.b)


  @property
  def weights(self):
    return [self.W, self.b]

In [3]:
class mySequential():
  def __init__(self, layers):
    self.layers = layers

  def __call__(self,inputs):
    x =  inputs
    for l in self.layers:
      x = l(x)
    return x

  @property
  def weights(self):
    weights = []
    for l in self.layers:
      weights += l.weights
    return weights

In [6]:
model = mySequential([
    myDense(input_size = 28*28, output_size = 512, activation = tf.nn.relu),
    myDense(input_size = 512, output_size = 10, activation = tf.nn.softmax)
])

In [8]:
import math 

class BatchGenerator():
  def __init__(self, images, labels, batch_size = 128):
    assert len(images) == len(labels), "Length of images is not equal of length of labels"
    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = math.ceil(len(images)/batch_size)

  def next(self):
    images = self.images[self.index : self.index + self.batch_size]
    labels = self.labels[self.index : self.index + self.batch_size]
    self.index += self.batch_size
    return images, labels


In [9]:
def training_step(model, image_batch, label_batch):
  with tf.GradientTape() as tape:
    predictions = model(image_batch)
    per_sample_loss = tf.keras.losses.sparse_categorical_crossentropy(label_batch, predictions)
    avarage_loss = tf.reduce_mean(per_sample_loss)
    gradients = tape.gradient(avarage_loss, model.weights)
    update_weights(gradients, model.weights)
  return avarage_loss

alpha = 1e-3

def update_weights(gradients, weights):
  for g,w in zip(gradients, weights):
    w.assign_sub(g*alpha)

#wartość straty wyświetlic, ciezej bedzie z accuracy

#metoda fit
